In [50]:
import torch

In [51]:
input_size = 4
batch_size = 1
hidden_size = 4
seq_len = 5
num_layers = 1
num_class = 4

In [52]:
# Data preparation
idx2char = ['e', 'h', 'l', 'o']
x_data = [1, 0, 2, 2, 3]
y_data = [3, 1, 2, 3, 2]

one_hot_lookup = [[1, 0, 0, 0],
                  [0, 1, 0, 0],
                  [0, 0, 1, 0],
                  [0, 0, 0, 1]]

x_one_hot = [one_hot_lookup[x] for x in x_data]

inputs = torch.Tensor(x_one_hot).view(-1, batch_size, input_size)
labels = torch.LongTensor(y_data).view(-1,1) # size (seqLen, 1)

In [53]:
class Model(torch.nn.Module):
    def __init__(self, input_size, hidden_size, batch_size):
        super(Model, self).__init__()
        self.batch_size = batch_size
        self.input_size = input_size
        self.hidden_size = hidden_size
        self.rnncell = torch.nn.RNNCell(input_size=self.input_size, hidden_size=self.hidden_size)

    def forward(self, input, hidden):
        hidden = self.rnncell(input, hidden)
        return hidden
    
    def init_hidden(self):
        return torch.zeros(self.batch_size, self.hidden_size)
    
net = Model(input_size, hidden_size, batch_size)

In [54]:
criterion = torch.nn.CrossEntropyLoss() # 结合了softmax 激活函数 和 负对数似然损失（Negative Log Likelihood, NLL），计算预测值和目标值之间的差异
optimizer = torch.optim.Adam(net.parameters(), lr=0.001) # lr学习率，优化器每次更新参数的步长大小

In [55]:
# RNN cell example
for epoch in range(15):
    loss = 0
    optimizer.zero_grad()
    hidden = net.init_hidden()
    print('Predicted string:', end='')
    for input, label in zip(inputs, labels):
        hidden = net(input, hidden)
        print()
        loss += criterion(hidden, label)
        _, idx = hidden.max(dim=1)
        print(idx2char[idx.item()], end='')
    loss.backward()
    optimizer.step()
    print(', Epoch [%d/15] loss=%.4f' % (epoch+1, loss.item()))

Predicted string:
o
o
o
o
o, Epoch [1/15] loss=6.4354
Predicted string:
o
o
o
o
o, Epoch [2/15] loss=6.4218
Predicted string:
o
o
o
o
o, Epoch [3/15] loss=6.4083
Predicted string:
o
o
o
o
o, Epoch [4/15] loss=6.3948
Predicted string:
o
o
o
o
o, Epoch [5/15] loss=6.3813
Predicted string:
o
o
o
o
o, Epoch [6/15] loss=6.3679
Predicted string:
o
o
o
o
o, Epoch [7/15] loss=6.3545
Predicted string:
o
o
o
o
o, Epoch [8/15] loss=6.3411
Predicted string:
o
o
o
o
o, Epoch [9/15] loss=6.3277
Predicted string:
o
o
o
o
o, Epoch [10/15] loss=6.3144
Predicted string:
o
o
o
o
o, Epoch [11/15] loss=6.3012
Predicted string:
o
o
o
o
o, Epoch [12/15] loss=6.2879
Predicted string:
o
o
o
o
o, Epoch [13/15] loss=6.2748
Predicted string:
o
o
o
o
o, Epoch [14/15] loss=6.2616
Predicted string:
o
o
o
o
o, Epoch [15/15] loss=6.2485


In [56]:
# RNN example
class Model(torch.nn.Module):
    def __init__(self, input_size, hidden_size, batch_size, num_layers=1):
        super(Model, self).__init__()
        self.num_layers = num_layers
        self.batch_size = batch_size
        self.input_size = input_size
        self.hidden_size = hidden_size
        self.rnn = torch.nn.RNN(input_size=self.input_size, hidden_size=self.hidden_size, num_layers=self.num_layers)

    def forward(self, input):
        hidden = torch.zeros(self.num_layers, self.batch_size, self.hidden_size)
        out, _ = self.rnn(input, hidden)
        return out.view(-1, self.hidden_size)

net = Model(input_size, hidden_size, batch_size, num_layers)

# 损失函数和优化器
criterion = torch.nn.CrossEntropyLoss()
optimizer = torch.optim.Adam(net.parameters(), lr=0.1)

for epoch in range(100):
    optimizer.zero_grad()
    outputs = net(inputs)
    labels = torch.LongTensor(y_data)
    loss = criterion(outputs, labels)
    loss.backward()
    optimizer.step()

    _, index = outputs.max(dim=1)  # max函数得到预测的概率最大的值和最大值的索引，取dim=1说明得到的是索引值，dim=0是最大值
    index = index.data.numpy()
    print('Predicted: ', ''.join([idx2char[x] for x in index]), end='')
    print(', Epoch [%d/15] loss = %.3f' % (epoch + 1, loss.item()))

Predicted:  ohhhh, Epoch [1/15] loss = 1.427
Predicted:  ohlll, Epoch [2/15] loss = 1.054
Predicted:  ohlll, Epoch [3/15] loss = 0.908
Predicted:  ohlll, Epoch [4/15] loss = 0.854
Predicted:  ohlll, Epoch [5/15] loss = 0.805
Predicted:  ohlll, Epoch [6/15] loss = 0.755
Predicted:  ohlll, Epoch [7/15] loss = 0.704
Predicted:  ohlol, Epoch [8/15] loss = 0.655
Predicted:  ohlol, Epoch [9/15] loss = 0.612
Predicted:  ohlol, Epoch [10/15] loss = 0.581
Predicted:  ohlol, Epoch [11/15] loss = 0.555
Predicted:  ohlol, Epoch [12/15] loss = 0.531
Predicted:  ohlol, Epoch [13/15] loss = 0.506
Predicted:  ohlol, Epoch [14/15] loss = 0.479
Predicted:  ohlol, Epoch [15/15] loss = 0.452
Predicted:  ohlol, Epoch [16/15] loss = 0.430
Predicted:  ohlol, Epoch [17/15] loss = 0.415
Predicted:  ohlol, Epoch [18/15] loss = 0.404
Predicted:  ohlol, Epoch [19/15] loss = 0.397
Predicted:  ohlol, Epoch [20/15] loss = 0.392
Predicted:  ohlol, Epoch [21/15] loss = 0.387
Predicted:  ohlol, Epoch [22/15] loss = 0.3

In [ ]:
embedding_size = 10

inputs = torch.LongTensor(x_data)
labels = torch.LongTensor(y_data)

class Model(torch.nn.Module):
    def __init__(self):
        super(Model, self).__init__()
        self.emb = torch.nn.Embedding(input_size, embedding_size)
        self.rnn = torch.nn.RNN(input_size=input_size, hidden_size=hidden_size, num_layers=num_layers)